---
# **HOW TO RUN**
---
When running, create the folder ```english``` that contains the ```.txt files``` with English query sentences to execute.

---
# **IMPORT**
---

In [1]:
import os
import io
from embeding import *
from vector_store import *
import csv
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

/home/alex/miniconda3/envs/ai_challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
qdrant_url = "http://localhost:6333"
token = ""

vector_space = QdrantVectorSpace(
    qdrant_url=qdrant_url,
    token=token,
    collection_name="run",
    snapshot=True
)
vector_space.load_snapshot("../weights/snapshot/qdrant_snapshot.snapshot")

Snapshot uploaded successfully.
Response: {'result': True, 'status': 'ok', 'time': 19.445680875}


In [8]:
model = JinaTextEmbeding('../weights/jina-clip-v1')

---
# **SETTING BEFORE RUNNING**
---

In [9]:
# Folder path
folder_sum = "run"
folder_input = os.path.join(folder_sum, "english")
folder_output = os.path.join(folder_sum, "result")
folder_check = os.path.join(folder_sum, "check")

os.makedirs(folder_output, exist_ok=True)
os.makedirs(folder_check, exist_ok=True)

troll_video = ["L09_V009", "L06_V003"]

---
# **RUNNING**
---

In [31]:
data = []
file_names = os.listdir(folder_input)
for each in file_names:
    file_path = os.path.join(folder_input, each)
    with open(file_path, 'r') as f:
        data.append(f.read())

In [32]:
result = model.inference(data, norm_embeds=True)

In [33]:
images_solve = {}

for each_embedding, each_file in zip(result[0], file_names):
    answer = vector_space.query_by_text(each_embedding, top_k=100)
    images_solve[each_file] = []
    # Result
    file_output = os.path.join(folder_output, each_file)
    file_output = file_output.replace(".txt", ".csv")

    # Check
    file_check = os.path.join(folder_check, each_file)
    file_check = file_check.replace(".txt", ".csv")

    with open(file_output, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        for scored_point in answer:
            video_id = scored_point.payload.get('video_id', '')
            frame_id = scored_point.payload.get('frame_id', '')
            images_solve[each_file].append((video_id, frame_id))
            if video_id in troll_video:
                frame_id = str(int(frame_id) * 25 / 30)
            csvwriter.writerow([video_id, round(float(frame_id))])

    with open(file_check, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Video ID', 'Frame ID', 'BTC Frame ID', 'Time'])
        
        for scored_point in answer:
            video_id = scored_point.payload.get('video_id', '')
            frame_id = scored_point.payload.get('frame_id', '')

            index_time = int(frame_id) / 25
            minute = index_time // 60
            second = index_time % 60
            index_time = f"{int(minute)}min{second:.2f}s"

            btc_frame_id = frame_id
            if video_id in troll_video:
                btc_frame_id = int(frame_id) * 25 / 30
                
            csvwriter.writerow([video_id, frame_id, btc_frame_id, index_time])

---
# **ADD RESULT FOR QA**
---

### **- Add**

In [6]:
# Add column with the same value to the file
input_file = ""
content = ""
df = pd.read_csv(input_file, header=None)
df[""] = content
df.to_csv(input_file, index=False, header=None)

### **- Remove**

In [32]:
# Delete the last
input_file = ""
df = pd.read_csv(input_file, header=None)
df = df.iloc[:, :-1]
df.to_csv(input_file, index=False, header=None)